In [ ]:
import efel
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import scipy
from scipy.signal import find_peaks
from scipy import signal
from scipy import stats
from scipy.optimize import curve_fit

In [ ]:
#use neo to import either voltage or current clamp data in the correct, scaled units!
def load_neo_file(file_name, **kwargs):
    #this code takes the file and returns segments that can be easily broken out into time and either voltage or amps, which this block of code automatically computes
    import neo
    reader = neo.io.get_io(file_name)
    blocks = reader.read(**kwargs)
    new_blocks = []
    for bl in blocks:
        new_segments = []
        for seg in bl.segments:
            traces = []
            count_traces = 0
            analogsignals = seg.analogsignals

            for sig in analogsignals:
                traces.append({})
                traces[count_traces]['T'] = sig.times.rescale('ms').magnitude
                #need to write an if statement here for conversion
                try:
                    traces[count_traces]['A'] = sig.rescale('pA').magnitude
                except:
                    traces[count_traces]['V'] = sig.rescale('mV').magnitude
                count_traces += 1
            new_segments.append(traces)
        #new_blocks.append(efel_segments)
    return new_segments

#analyze an indiviudal feature for batch analysis from the efel package. This function will return a table with the feature values for each trace in the file.
def analyze_feature(path, feature):
    #####
    #this code takes the path to a directory and the eFEL feature, which are well documented (https://github.com/BlueBrain/eFEL), and returns a dataframe of the data for each trace
    table2 = []
    os.chdir(path)
    for filename in os.listdir():
        table = pd.DataFrame(columns=[feature])     #create a table that has columns with the name you want
        table.name = feature                        #the tables name
        if filename.endswith(".axgd") or filename.endswith(".axgx"):    #check for the filetype
            [traces] = efel.io.load_neo_file(filename, stim_start=500, stim_end=1500)    #load the trace, and define stim start and stop
            for data in traces:    #loop through these guys
                #table.rename(columns={feature:filename}, inplace=True) #renaming the columns with the correct file !
                feature_values = efel.getFeatureValues(data, [feature], raise_warnings=None)[0]  #this is the feature extraction
                if feature_values[feature] is not None:
                    # Define the parameters for detection
                    efel.api.setThreshold(-10) # Voltage threshold for detection
                    efel.api.setDerivativeThreshold(10) # dV/dt threshold for detection
                    efel.setIntSetting('strict_stiminterval', True) #this is an important param to force eFEL to only analysis the desired time window
                    length = len(table)
                    table.loc[length, feature] = feature_values[feature][0]

                else:
                    efel.api.setThreshold(-10) # Voltage threshold for detection
                    efel.api.setDerivativeThreshold(10) # dV/dt threshold for detection
                    efel.setIntSetting('strict_stiminterval', True)
                    length = len(table)
                    table.loc[length, feature] = feature_values[feature]

            table2.append(table)
            df_concat = pd.concat(table2, axis=1)
            table.rename(columns={feature:filename}, inplace=True) #renaming the columns with the correct file !
            #block of code to combine all of the generated excel workbooks into a single workbook
#             df_concat.to_excel(feature + 'master_file.xlsx', index=False)
    Current_injected = np.linspace(-100.0, 500.0, num=61)
    table2 = df_concat.assign(Current_injected=Current_injected)
    return(df_concat)

In [ ]:
cc_path = os.getcwd()

In [ ]:
#these code are well documented on efels github page. With strong documentation. These code are only used for the evoked spikes. All spontaneous firing code can be found in the "Spontaenous firing analysis" folder. Here we use a steeper threshold for the detection of spikes. 
#these code return excel sheets that will be saved in the path that you are currently in with all of the data. I have included a few example files to show you how to use the code.
os.chdir(cc_path)
f_i = analyze_feature(cc_path, feature='Spikecount_stimint')